# Redis NoSQL Database and Pub/Sub

In [174]:
import redis 
red_server = redis.StrictRedis(host="localhost", port=6379, db=0)

## String Data Structure / StringCommands

### Add Values

In [ ]:
red_server.set("name","Ali")
red_server.set("surname","Kalbaliyev")
red_server.set("age","24")

### Increase and Decrease integer value

In [176]:
red_server.incr("age") # default 1

25

In [177]:
red_server.incr("age",10)

35

In [178]:
red_server.decr("age") # default 1

34

In [179]:
red_server.decr("age",10)

24

### Multiple Set Keys

In [ ]:
red_server.mset({
    "firstname":"Ali",
    "lastname":"Kalbaliyev",
    "position":"Data Scientist"
})

### Get values from Redis

In [181]:
red_server.type("surname") # show redis type of key

b'string'

In [182]:
for key in ["name","surname","age"]:
    val=red_server.get(key)
    print(f"Type: {red_server.type(key)}, Key:value | {key} => {val}")
print("------------------------------------------------------")
for key in ["firstname","lastname","position"]:
    val=red_server.get(key)
    print(f"Type: {red_server.type(key)}, Key:value | {key} => {val}")

Type: b'string', Key:value | name => b'Ali'
Type: b'string', Key:value | surname => b'Kalbaliyev'
Type: b'string', Key:value | age => b'24'
------------------------------------------------------
Type: b'string', Key:value | firstname => b'Ali'
Type: b'string', Key:value | lastname => b'Kalbaliyev'
Type: b'string', Key:value | position => b'Data Scientist'


In [183]:
red_server.keys() # get all keys' list

[b'position', b'surname', b'firstname', b'name', b'lastname', b'age']

In [184]:
for key in red_server.keys():
    val=red_server.get(key)
    print(f"Type: {red_server.type(key)}, Key:value | {key} => {val}")

Type: b'string', Key:value | b'position' => b'Data Scientist'
Type: b'string', Key:value | b'surname' => b'Kalbaliyev'
Type: b'string', Key:value | b'firstname' => b'Ali'
Type: b'string', Key:value | b'name' => b'Ali'
Type: b'string', Key:value | b'lastname' => b'Kalbaliyev'
Type: b'string', Key:value | b'age' => b'24'


### Multiple Get Values

In [185]:
red_server.mget(red_server.keys())

[b'Data Scientist', b'Kalbaliyev', b'Ali', b'Ali', b'Kalbaliyev', b'24']

In [186]:
dict(zip(red_server.keys(),red_server.mget(red_server.keys())))

{b'position': b'Data Scientist',
 b'surname': b'Kalbaliyev',
 b'firstname': b'Ali',
 b'name': b'Ali',
 b'lastname': b'Kalbaliyev',
 b'age': b'24'}

### Exists

In [187]:
red_server.exists("name") # True=1, False=0

1

In [188]:
red_server.exists("url") # True=1, False=0

0

### Set Expired Key

In [189]:
import time

red_server.set("token","EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS")
red_server.expire("token",5)

while red_server.get("token"):
    print(f"Value :{red_server.get('token')}")
    time.sleep(1)
else:
    print("Expired key")

Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
Expired key


In [190]:
red_server.get("token") # output is nothing

### Change Expired to Persistent ( with exists method )

In [191]:
import time

red_server.set("token","EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS")
red_server.expire("token",5)
i=1

while red_server.exists("token"):
    print(f"{i} seconds | Value :{red_server.get('token')}")
    
    if i==3:
        red_server.persist("token")
    elif i==10:
        print("Key is already persistent")
        break
    i+=1
    time.sleep(1)
else:
    print("Expired key")

1 seconds | Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
2 seconds | Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
3 seconds | Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
4 seconds | Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
5 seconds | Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
6 seconds | Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
7 seconds | Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
8 seconds | Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
9 seconds | Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
10 seconds | Value :b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'
Key is already persistent


In [192]:
red_server.get("token")

b'EUWDH72GF942GS2FH29F83WU9WF382HBD219uwe28GS72NDS'

### Delete Key

In [193]:
red_server.delete("token")
red_server.get("token") # After Deleting Output is nothing

### Append string

In [194]:
print("Value: ",red_server.get("position"))
red_server.append("position"," | ML Engineer")
print("Value: ",red_server.get("position"))

Value:  b'Data Scientist'
Value:  b'Data Scientist | ML Engineer'


### Rename Keys

In [195]:
print("Keys: ",red_server.keys())
red_server.rename("age","ageofuser")
print("Keys: ",red_server.keys())

Keys:  [b'position', b'surname', b'firstname', b'name', b'lastname', b'age']
Keys:  [b'position', b'surname', b'firstname', b'name', b'lastname', b'ageofuser']


### Create Dict Value as Dict on String Data Structure

In [450]:
import json

data = {'fruits':['apple','mango','banana'], 'veggies':['spinach','onion','beans']}
red_server.set('data', json.dumps(data))
json.loads(red_server.get('data'))

{'fruits': ['apple', 'mango', 'banana'],
 'veggies': ['spinach', 'onion', 'beans']}

### Delete All data on db

In [ ]:
red_server.flushdb()
red_server.keys()

## List Data Structure / ListCommands

### Add Values like DEQUE

In [ ]:
red_server.lpush("mylist",70) #Left Append =  [70]

In [ ]:
red_server.lpush("mylist",200,80) #Left Append = [200, 70] -> [80,200,70]

In [ ]:
red_server.rpush("mylist",99,11,55) #Right Append = [80,200,70,99] -> [80,200,70,99,11] -> [80,200,70,99,11,55]

In [ ]:
red_server.lpush("mylist2",50,"Ali")

In [ ]:
red_server.linsert("mylist2","before","50","Kalbaliev") # add value Kalbaliev , before 50 

### Lenght Of List

In [323]:
red_server.llen("mylist")

6

### Get Values From List

##### Indexing

In [324]:
red_server.lindex("mylist2",0)

b'Ali'

##### Slicing

In [325]:
red_server.lrange("mylist",0,2)

[b'80', b'200', b'70']

In [326]:
red_server.lrange("mylist",0,-1)

[b'80', b'200', b'70', b'99', b'11', b'55']

In [327]:
red_server.lrange("mylist2",0,(red_server.llen("mylist2")-1))

[b'Ali', b'Kalbaliev', b'50']

### Overwrite Value of List

In [ ]:
red_server.lset("mylist2",0,"Data Scientist")

In [329]:
red_server.lrange("mylist2",0,(red_server.llen("mylist2")-1))

[b'Data Scientist', b'Kalbaliev', b'50']

### Delete Value like DEQUE

In [330]:
red_server.lpop("mylist") # Left Pop - Delete

b'80'

In [331]:
red_server.rpop("mylist") # Right Pop - Delete

b'55'

In [332]:
red_server.lrange("mylist",0,-1)

[b'200', b'70', b'99', b'11']

### Delete Value for Value

In [ ]:
red_server.lrem("mylist",1,"70")

In [334]:
red_server.lrange("mylist",0,-1)

[b'200', b'99', b'11']

### Delete All data on db

In [ ]:
red_server.flushdb()
red_server.keys()

## Dictionary Data Structure / HashCommands

### One Value Add to Hash

In [ ]:
red_server.hset("student1","name","Ali")
red_server.hset("student1","surname","Kalbaliev")
red_server.hset("student1","age","24")

In [436]:
red_server.hget("student1","surname") # Get one key's value

b'Kalbaliev'

In [437]:
red_server.hmget("student1",["name","surname"])  # Get Multiple keys' values

[b'Ali', b'Kalbaliev']

In [438]:
red_server.hgetall("student1") # Get All Values

{b'name': b'Ali', b'surname': b'Kalbaliev', b'age': b'24'}

### Multiple Values Add to Hash

In [ ]:
red_server.hmset("student1",
                 {"name":"Ali",
                  "surname":"Kalbaliyev",
                  "age":24,
                  "gender":"Male",
                  "studentStatus":1, # instead of True
                  "Courses":'["Redis","Docker","Apache Kafka"]'})

In [440]:
red_server.hgetall("student1") # Get All Values

{b'name': b'Ali',
 b'surname': b'Kalbaliyev',
 b'age': b'24',
 b'gender': b'Male',
 b'studentStatus': b'1',
 b'Courses': b'["Redis","Docker","Apache Kafka"]'}

In [441]:
red_server.hget("student1","Courses")

b'["Redis","Docker","Apache Kafka"]'

##### Hash - Keys and Values

In [442]:
print("Keys: ",red_server.hkeys("student1"))
print("Values: ",red_server.hvals("student1"))

Keys:  [b'name', b'surname', b'age', b'gender', b'studentStatus', b'Courses']
Values:  [b'Ali', b'Kalbaliyev', b'24', b'Male', b'1', b'["Redis","Docker","Apache Kafka"]']


##### Hash - Exists

In [443]:
print(red_server.hexists("student1","Courses"))
print(red_server.hexists("student1","Numbers"))

True
False


##### Hash - Len

In [444]:
print(red_server.hlen("student1")) # Elements size

6


### Delete Values From Hash

In [ ]:
red_server.hdel("student1","studentStatus")

In [446]:
red_server.hgetall("student1")

{b'name': b'Ali',
 b'surname': b'Kalbaliyev',
 b'age': b'24',
 b'gender': b'Male',
 b'Courses': b'["Redis","Docker","Apache Kafka"]'}

### Extra: Decode Bytes to Dict and List From "Added String Dict and String List"

In [ ]:
red_server.hset("student1","education",'{"university": "ASOIU","degree": "Bachelor","years": 4}')

In [452]:
red_server.hgetall("student1")

{b'name': b'Ali',
 b'surname': b'Kalbaliyev',
 b'age': b'24',
 b'gender': b'Male',
 b'Courses': b'["Redis","Docker","Apache Kafka"]',
 b'education': b'{"university": "ASOIU","degree": "Bachelor","years": 4}'}

In [449]:
import ast

decoded1 = red_server.hget("student1","Courses").decode("UTF-8")
courses_list= ast.literal_eval(decoded1)


decoded2 = red_server.hget("student1","education").decode("UTF-8")
education_dict= ast.literal_eval(decoded2)

print("Courses List: ",courses_list , "Type: ",type(courses_list))
print("Education Dict: ",education_dict , "Type: ",type(education_dict))

Courses List:  ['Redis', 'Docker', 'Apache Kafka'] Type:  <class 'list'>
Education Dict:  {'university': 'ASOIU', 'degree': 'Bachelor', 'years': 4} Type:  <class 'dict'>


## Pub/Sub - Publisher Subsriber ( Messaging Broker )

In [456]:
import threading

In [110]:
class Listener(threading.Thread):
    def __init__(self,red_server,channels):
        threading.Thread.__init__(self)
        
        self.redis = red_server
        self.pubsub = self.redis.pubsub() 
        self.pubsub.subscribe(channels)

    def work(self,message_props):
        if message_props['type']=='message':
            print("Channel: ",message_props['channel'],
                  "\nMessage:", message_props["data"])
        else:
            print(f"Subscribed to {message_props['channel']}")

    def run(self):
        for message_props in self.pubsub.listen():       
            
            if  message_props["data"] == bytes("KILL", 'utf-8'):
                self.pubsub.unsubscribe()
                print ("Unsubscribed")
                break
            else:
                self.work(message_props)

In [111]:
client = Listener(red_server,["mychannel-1","mychannel-2"])
client.start()

Subscribed to b'mychannel-1'
Subscribed to b'mychannel-2'


#### Channel 1 - Publishing

In [112]:
print(red_server.publish("mychannel-1","My first message"))

Channel: 1 b'mychannel-1' 
Message: b'My first message'



In [113]:
red_server.publish("mychannel-1","My second message")

Channel: 

1

 b'mychannel-1' 
Message: b'My second message'


#### Channel 2 - Publishing

In [114]:
red_server.publish("mychannel-2","My first message")
red_server.publish("mychannel-2","My second message")
red_server.publish("mychannel-2","KILL")

Channel:  b'mychannel-2' 
Message: b'My first message'
Channel:  b'mychannel-2' 
Message:

1

 b'My second message'
Unsubscribed


### After Unsubscribe

In [115]:
red_server.publish("mychannel-2","My second message")

0